In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("goodreads_data.csv", engine="python", quotechar='"', on_bad_lines="skip");
df.shape

(10000, 8)

In [4]:
ndf = df.dropna(axis=0, subset=["Author", "Description", "Genres", 'Book'])
ndf.duplicated(keep='last', subset=["Book"])

ndf.shape

(9923, 8)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

def eliminate_stop_words(text):
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    word_tokens = word_tokenize(text)
    filtered_word = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_word.append(word)

    filtered_text = ' '.join(filtered_word)
    print(filtered_text)
    return filtered_text 

ndf['cleaned_genres'] = ndf['Genres'].apply(eliminate_stop_words)
#ndf['cleaned_combined'] = ndf['cleaned_desc'] + ' ' + ndf['Genres'] + ' ' + ndf['Author']
ndf['cleaned_combined'] = ndf['cleaned_genres'] + ' ' + ndf['Author']

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tf = TfidfVectorizer(max_features=5000).fit(ndf['cleaned_combined'])
tf_matrix = tf.fit_transform(ndf['cleaned_combined'])

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tf_matrix, tf_matrix)
indices = pd.Series(ndf.index, index=ndf['Book']).drop_duplicates()

def recommend_books(title, top_n = 5):
    idx = indices[title]
    sim = list(enumerate(cosine_sim[idx]))
    sim = sorted(sim, key=lambda x: x[1],reverse=True)
    sim = sim[1:top_n+1]
    books_idx = [i[0] for i in sim if i[0] != idx]
    return df[['Book', 'Author']].iloc[books_idx]

#recommend_books("The Brothers Karamazov", 10)
recommend_books("Dr. Jekyll and Mr. Hyde", 10)
#recommend_books("Ubik", 10)

,Book,Author
1201,Antigone; Oedipus the King; Electra,Sophocles
7217,"Lover Avenged (Black Dagger Brotherhood, #7)",J.R. Ward
3473,The Known World,Edward P. Jones
294,Treasure Island,Robert Louis Stevenson
5625,The Master of Go,Yasunari Kawabata
2151,We Disappear,Scott Heim
8862,Tree Oracle - The True Story of Carmen Sylvia,Laural Virtues Wauters
5913,Easy Money Management: A Give and Save 3-6-5 A...,Laurick Ingram
6517,All the Light We Cannot See,Anthony Doerr
8010,The Book of Revelation,Rupert Thomson
